In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
import math
from scipy import sparse
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy

In [2]:
PROBLEM_SIZE = 100
NUM_SETS = 500

In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

 ON THIS PROBLEM I TRIED TO IMPLEMENT THE DIFFERENT METHODS THAT WE STUDIED DURING THE LECTURES ABOUT HILL CLIMBING. I USED 3 FITNESS FUNCTION TO MISURE THE QUALITY OF A STATE. THE FIRST FITNESS FUNCTION AND THE SECOND ARE USED IN THE MOST OF METHODS. THE THIRD IS ONLY USED IN SUIMULATED ANNEALING WHERE WE HAVE TO SEARCH A SOLUTION THAT CONSIDER ALSO CASES IN WHICH COULD BE  MANY PROBLEMS. IN THE IMPLEMENTATION I USED FITNESS1 AND FITNESS3 BUT I ALSO TRIED IN THE PREVIOUS DAYS TO IMPLEMENTO FITNESS2 AND FITNESS3. THE RESULT IN THE FIRT IMPLEMENTATION IS BETTER FOR SIMPLY HILL CLIMBING. ON THE CONTRARY, IN THE SECOND WE HAVE AN IMPLEMENTATION THAT IS BETTER FOR SIMULATED ANNELING.

In [4]:
sets=make_set_covering_problem(100,100,.3)
sets_sparsi=tuple(sets.toarray())
cost_candidata=30
temperature=1000 #I assume that i have a high temperature, it's not the best solution but it is admissible. It permitts a large space of exploration
def fitness1(state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [sets_sparsi[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

def fitness2(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets_sparsi[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost
def fitness3(state):
    cost=sum(state)
    cost_tweaked=sum(tweak(state))
    #valid=np.sum(reduce(np.logical_or,[sets_sparsi[i] for i, t in enumerate(state) if t],np.array([False for _ in range(PROBLEM_SIZE)]),))
    valid = np.all(reduce(np.logical_or,[sets_sparsi[i] for i, t in enumerate(state) if t],np.array([False for _ in range(PROBLEM_SIZE)]),))
    if valid:
        return -cost

    delta_cost = cost - cost_tweaked #delta that we used in the definition for Simulated Annealing
    probability = math.exp(delta_cost / temperature) # value of p used on Simulated Annealing

    if random() < probability:
        return -cost
    else:
        return -float('inf')
    return valid, -cost


In [5]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

def LocalSearch(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    current_state=copy(new_state)

    if fitness2(current_state)>=fitness2(new_state):
      return current_state

    return new_state


In [6]:
##############################################
#    HILL CLIMBING WITH                      #  ### CON LA FITNESS1 è il miglior metodo mentre con la FITNESS2 il miglior metodo è il SIMULATED ANNEALING
#    SIMPLE HILL CLIMBING                    #
##############################################
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
counter=0
for step in range(10_000):#This is the normal execution used for HILL CLIMBING
    new_state = tweak(current_state)
    if fitness2(new_state) >= fitness2(current_state): #We are calculating the quality of solution with fitness function that is for SIMULATED ANNEALING
        current_state = new_state

        counter+=1


print(f"The final counter to reach the Hill Climbing with Simple Hill Climbing is {counter}")

The final counter to reach the Hill Climbing with Simple Hill Climbing is 15


In [7]:
##############################################
#    HILL CLIMBING WITH                      #
#    SIMULATED ANNEALING                     #
##############################################
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
counter=0
for step in range(10_000):#This is the normal execution used for HILL CLIMBING
    new_state = tweak(current_state)
    if fitness3(new_state) >= fitness3(current_state): #We are calculating the quality of solution with fitness function that is for SIMULATED ANNEALING
        current_state = new_state

        counter+=1


print(f"The final counter to reach the Hill Climbing with Simulated Annealing is {counter}")

The final counter to reach the Hill Climbing with Simulated Annealing is 20


In [8]:
##############################################
#    HILL CLIMBING WITH                      #
#    ITERATED LOCAL SEARCH                   #
##############################################
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
counter=0
#stato_perturbato = perturbation(current_state)
for step in range(10_000):#This is the normal execution used for HILL CLIMBING
    stato_perturbato = LocalSearch(current_state)
    new_state=tweak(stato_perturbato)
    if fitness1(new_state) >= fitness1(current_state): #We are calculating the quality of solution with fitness function that is for SIMULATED ANNEALING
        current_state = new_state

        counter+=1


print(f"The final counter to reach the Hill Climbing with Iterated Local Search is {counter}")

The final counter to reach the Hill Climbing with Iterated Local Search is 96


In [9]:
##############################################
#    HILL CLIMBING WITH                      #
#    TABU SEARCH                             #
##############################################


current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
tabu=[]
counter=0
best_state=current_state
for step in range(10_000):

    tmp= [tweak(current_state) for _ in range(NUM_SETS)]
    best = None

    for neighbor in tmp:
        if neighbor not in tabu:
            if best is None or fitness1(neighbor) > fitness1(best):
                best = neighbor

    if best is not None:
        current_solution = best
        tabu.append(current_solution)

        if fitness1(current_solution) > fitness1(best_state):
            best_state = current_solution

    counter += 1
    tabu.pop(0)

print(f"The final counter to reach the Hill Climbing with Tabu Search is {counter}")



The final counter to reach the Hill Climbing with Tabu Search is 10000
